In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import medfilt

from numpy.fft import fft, fftshift, fftfreq
from matplotlib import pyplot as plt
from pathlib import Path
import importlib

In [28]:
import helpers
importlib.reload(helpers)

# Driver
class Ex:
    def __init__(self, name: str, num_notes: int, spec_thresh: float, bpm: int, max_num_notes_per_beat: int):
        self.name = name
        self.num_notes = num_notes
        self.spec_thresh = spec_thresh
        self.bpm = bpm
        self.max_num_notes_per_beat = max_num_notes_per_beat
        self.min_time_between = 60 / (self.bpm * max_num_notes_per_beat)

exerciseNoteCounts = [
    Ex("ex1WholeMod.mp4", num_notes=90, spec_thresh=.15, bpm=80, max_num_notes_per_beat=2),
    Ex("ex1WholeModF.mp4", num_notes=90, spec_thresh=.15, bpm=120, max_num_notes_per_beat=2),
    Ex("ex2WholeMod.mp4", num_notes=49, spec_thresh=.15, bpm=80, max_num_notes_per_beat=1),
    Ex("ex3WholeMod.mp4", num_notes=145, spec_thresh=.15, bpm=100, max_num_notes_per_beat=2),
    Ex("ex3WholeModF.mp4", num_notes=145, spec_thresh=.15, bpm=130, max_num_notes_per_beat=2),
    Ex("ex4WholeMod.mp4", num_notes=102, spec_thresh=.15, bpm=90, max_num_notes_per_beat=4),
    Ex("ex4WholeModF.mp4", num_notes=102, spec_thresh=.05, bpm=120, max_num_notes_per_beat=2),
    Ex("ex5WholeMod.mp4", num_notes=133, spec_thresh=.15, bpm=64, max_num_notes_per_beat=2),
    Ex("ex5WholeModF.mp4", num_notes=133, spec_thresh=.15, bpm=86, max_num_notes_per_beat=2),
    Ex("ex6WholeMod.mp4", num_notes=118, spec_thresh=.15, bpm=70, max_num_notes_per_beat=4),
    Ex("ex6WholeModF.mp4", num_notes=118, spec_thresh=.15, bpm=90, max_num_notes_per_beat=4),
    Ex("ex7WholeMod.mp4", num_notes=86, spec_thresh=.15, bpm=70, max_num_notes_per_beat=2),
    Ex("ex8WholeMod.mp4", num_notes=112, spec_thresh=.15, bpm=55, max_num_notes_per_beat=4),
    Ex("ex8WholeModF.mp4", num_notes=112, spec_thresh=.15, bpm=80, max_num_notes_per_beat=4),
    Ex("ex9WholeMod.mp4", num_notes=121, spec_thresh=.15, bpm=100, max_num_notes_per_beat=4),
    Ex("ex10WholeMod.mp4", num_notes=102, spec_thresh=.15, bpm=120, max_num_notes_per_beat=3)
]

avg_artic_leng = 0
for ex in exerciseNoteCounts:
    exercise = ex.name
    note_count = ex.num_notes
    spec_thresh = ex.spec_thresh
    min_time_between = ex.min_time_between * .8

    ys, ts, sr = helpers.get_audio_data(f"exercises/{exercise}")
    ts_fr, fr_freq_amps, freq_bins = helpers.magnitude_spectrogram(ys, ts, sr)
    ts_fr, spec_flux = helpers.compute_spectral_flux(ts_fr, fr_freq_amps, sr)

    centroids = helpers.compute_spectral_centroid(time_frames=ts_fr, freq_bins=freq_bins, frame_freq_amps=fr_freq_amps)

    
    plt.title(exercise)
    plt.xlabel("Time (s)"); plt.ylabel("Spectral Flux")
    plt.plot(ts_fr, spec_flux, c="orange")
    img_path = Path.cwd() / "spec_flux_graphs" / f"{exercise} Spectral Flux.png"
    plt.savefig(img_path)
    plt.close()
    
    # onsets = helpers.detect_onsets(spec_flux, ts_fr, threshold=spec_thresh, min_time_between=min_time_between)
    onsets, sustains = helpers.detect_onsets_and_release(spec_flux=spec_flux, times=ts_fr, sr=sr, onset_thresh=spec_thresh, min_time_between=min_time_between)

    print_num_onset_detection_accuracy = True
    graph_onsets_sustains = True
    
    if print_num_onset_detection_accuracy:
        print(f"For exercise {exercise}, Note count Actual:\t{note_count}, Note count Detected: \t{len(onsets)} | Accuracy: \t{min(note_count / len(onsets), len(onsets) / note_count)}")

    if graph_onsets_sustains:
        artic_lens = [sustains[i] - onsets[i] for i in range(min(len(onsets), len(sustains)))]
        # if np.max(artic_lens) > 1:
        # print(f"For {exercise}, min articulation length: {np.min(artic_lens)}")
        # if np.max(artic_lens) > .1:
        #     print("==============================================================")
        #     print(f"For {exercise}, max articulation length: {np.max(artic_lens)}")
        print(f"For {exercise}, mean articulation length: {np.average(artic_lens)}")
        avg_artic_leng += np.average(artic_lens)

        onset_indic = np.zeros_like(ts_fr)
        sustain_indic = np.zeros_like(ts_fr)

        stem_top = np.max(centroids)

        # Set 1.0 at the closest time points where onsets/sustains occur
        for onset_time in onsets:
            idx = np.argmin(np.abs(ts_fr - onset_time))
            onset_indic[idx] = stem_top
            
        for sustain_time in sustains:
            idx = np.argmin(np.abs(ts_fr - sustain_time))
            sustain_indic[idx] = stem_top

        plt.title(f"{exercise} Onset + Sustain + Spectral Centroid")
        plt.xlim(10, 14)
        plt.xlabel("Times (s)")
        plt.stem(ts_fr, onset_indic, linefmt='--', markerfmt='pink', label='Onsets')
        plt.stem(ts_fr, sustain_indic, linefmt='--', markerfmt='red', label='Sustains')
        plt.legend()
        img_path = Path.cwd() / "centroids" / f"{exercise} centroid.png"
        plt.plot(ts_fr, centroids)
        plt.savefig(img_path)
        plt.close()

print(f"Average articulation window size is{avg_artic_leng / len(exerciseNoteCounts)}")

For exercise ex1WholeMod.mp4, Note count Actual:	90, Note count Detected: 	88 | Accuracy: 	0.9777777777777777
For ex1WholeMod.mp4, mean articulation length: 0.0329545454545455
For exercise ex1WholeModF.mp4, Note count Actual:	90, Note count Detected: 	90 | Accuracy: 	1.0
For ex1WholeModF.mp4, mean articulation length: 0.032888888888888926
For exercise ex2WholeMod.mp4, Note count Actual:	49, Note count Detected: 	46 | Accuracy: 	0.9387755102040817
For ex2WholeMod.mp4, mean articulation length: 0.031086956521739116
For exercise ex3WholeMod.mp4, Note count Actual:	145, Note count Detected: 	142 | Accuracy: 	0.9793103448275862
For ex3WholeMod.mp4, mean articulation length: 0.0379577464788733
For exercise ex3WholeModF.mp4, Note count Actual:	145, Note count Detected: 	141 | Accuracy: 	0.9724137931034482
For ex3WholeModF.mp4, mean articulation length: 0.036170212765957416
For exercise ex4WholeMod.mp4, Note count Actual:	102, Note count Detected: 	106 | Accuracy: 	0.9622641509433962
For ex4Wh

In [32]:
def optimize_spec_thresh():
    lowest, highest, num_tests = .01, .25, 100
    spec_thres_vals = np.linspace(lowest, highest, num_tests)
    ret = [] # the optimized thresholds for each exercise

    for ex in exerciseNoteCounts:
        exercise = ex.name
        note_count = ex.num_notes
        min_time_between = ex.min_time_between * .8

        num_onsets_detected = []
        accuracies = []

        ys, ts, sr = helpers.get_audio_data(f"exercises/{exercise}")
        ts_fr, fr_freq_amps, _ = helpers.magnitude_spectrogram(ys, ts, sr)
        ts_fr, spec_flux = helpers.compute_spectral_flux(ts_fr, fr_freq_amps, sr)

        for spec_thresh in spec_thres_vals:
            onsets, sustains = helpers.detect_onsets_and_release(spec_flux=spec_flux, times=ts_fr, sr=sr, onset_thresh=spec_thresh, sustain_thresh=.13, min_time_between=min_time_between)
            accuracy = min(note_count / len(onsets), len(onsets) / note_count)

            num_onsets_detected.append(len(onsets))
            accuracies.append(accuracy)
        
        plt.title(f"{exercise} Spectral Flux Threshold vs Number of Onsets Detected")
        plt.xlabel("Spectral Flux Threshold"); plt.ylabel("Number of Onsets Detected")
        plt.plot(spec_thres_vals, num_onsets_detected, label="Threshold vs Onsets")
        plt.axhline(y=note_count, color='limegreen', linestyle='--', linewidth=2, label="Correct # Onsets")
        plt.legend()
        img_path = Path.cwd() / "spec_thresh_opt" / "graphs" / f"{exercise}.png"
        plt.savefig(img_path)
        plt.close()

        opt_perf = max(accuracies)
        opt_perf_spec_thresh = [spec_thres_vals[i] for i, val in enumerate(accuracies) if val == opt_perf]
        with open("spec_thresh_opt/optimized_values.txt", "a") as f:
            f.write(f"{'=' * 30}\n")
            f.write(f"Best performing spectral thresholds at {opt_perf * 100}% accuracy for {exercise}:\n")
            f.write(f"{opt_perf_spec_thresh}\n\n")

        ret.append(opt_perf_spec_thresh[len(opt_perf_spec_thresh) // 2]) # choose the middle most successful output to maximize applicability
        
    return ret

def apply_optimized_spec_thresh(opts):
    for i, ex in enumerate(exerciseNoteCounts):
        ex.spec_thresh = opts[i]

opt_spec_thresh = optimize_spec_thresh()
apply_optimized_spec_thresh(opt_spec_thresh)

In [ ]:
def optimize_sustain_thresh_coeff():
    lowest, highest, num_tests = 0, .85, 100
    sustain_thresh_coeffs = np.linspace(lowest, highest, num_tests)
    ret = [] # the optimized coefficients for each exercise

    for ex in exerciseNoteCounts:
        exercise = ex.name
        note_count = ex.num_notes
        spec_thresh = ex.spec_thresh
        min_time_between = ex.min_time_between * .8

        sustain_saturations = []

        ys, ts, sr = helpers.get_audio_data(f"exercises/{exercise}")
        ts_fr, fr_freq_amps, _ = helpers.magnitude_spectrogram(ys, ts, sr)
        ts_fr, spec_flux = helpers.compute_spectral_flux(ts_fr, fr_freq_amps, sr)

        for coeff in sustain_thresh_coeffs:
            sustain_thresh = spec_thresh * coeff
            onsets, sustains = helpers.detect_onsets_and_release(spec_flux=spec_flux, times=ts_fr, sr=sr, onset_thresh=spec_thresh, sustain_thresh=sustain_thresh, min_time_between=min_time_between)

            sustain_saturations.append(len(sustains) / len(onsets))

        plt.title(f"{exercise} Coefficient for Sustain vs Onset Pairing Completeness")
        plt.xlabel("Coefficient for Sustain"); plt.ylabel("Onset Pairing Completeness")
        plt.plot(sustain_thresh_coeffs, sustain_saturations)
        img_path = Path.cwd() / "sustain_thresh_opt" / "graphs" / f"{exercise}.png"
        plt.savefig(img_path)
        plt.close()

        opt_perf_sustain_coeff = [sustain_thresh_coeffs[i] for i, val in enumerate(sustain_saturations) if val == 1]
        with open("sustain_thresh_opt/optimized_values.txt", "a") as f:
            f.write(f"{'=' * 30}\n")
            f.write(f"Minimum coefficient for complete onset + sustain pairing for {exercise}:\n")
            f.write(f"{np.min(opt_perf_sustain_coeff)}\n\n")

        ret.append(np.min(opt_perf_sustain_coeff)) # choose the middle most successful output to maximize applicability
        
    return ret

# for ex in exerciseNoteCounts:
#     print(f"for {ex.name}: {ex.spec_thresh}")

optimize_sustain_thresh_coeff() 
# Conclusion:
# The sustain threshold can be essentially as small a nonzero value as you'd like 
# So in practice can `sustain_thresh = onset_thresh * .01` should suffice 
# This is overall great news because it indicates that using spectral flux for 
# onset detection + sustain phase detection is a great approach, because
# there's a ton of granularity. The spectral flux goes down to essentially 0 (very accurate)
# Of course, this is probably partially due to smoothing, but still a good result 


[np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586),
 np.float64(0.008585858585858586)]